### Fine Tuning Task  

강아지, 머핀 이진 분류  
https://drive.google.com/file/d/1Q_KC2nTmGlX2PeINWwV63_kpTjnoVpiO/view?usp=sharing

In [1]:
from glob import glob
import os

root = './datasets/muffin/original/'

directories = glob(os.path.join(root,'*'))
dir_names = []

# root 경로 안 directory들의 이름 출력
for directory in directories:
    dir_names.append(directory[directory.rindex('\\') + 1:])

print(dir_names)

['chihuahua', 'muffin']


In [2]:
# 각 폴더 안 이미지 파일의 이름을 '폴더명+숫자(01, 02, ...).png'로 변경
for name in dir_names:
    for i, file_name in enumerate(os.listdir(os.path.join(root, name))):
        # 기존 파일명
        old_file = os.path.join(root + name + '/', file_name)

        # 변경 후 파일명
        new_file = os.path.join(root + name + '/', name + str(i + 1) + '.png')

        os.rename(old_file, new_file)

In [3]:
from glob import glob
import os

root = './datasets/muffin/test/'

directories = glob(os.path.join(root,'*'))
dir_names = []

# test 폴더도 동일한 과정 진행
for directory in directories:
    dir_names.append(directory[directory.rindex('\\') + 1:])

print(dir_names)

['chihuahua', 'muffin']


In [4]:
# 각 폴더 안 이미지 파일의 이름을 '폴더명+숫자(01, 02, ...).png'로 변경
for name in dir_names:
    for i, file_name in enumerate(os.listdir(os.path.join(root, name))):
        # 기존 파일명
        old_file = os.path.join(root + name + '/', file_name)

        # 변경 후 파일명
        new_file = os.path.join(root + name + '/', name + str(i + 1) + '.png')

        os.rename(old_file, new_file)

### original → Train, Validation 데이터 세트 분할

In [57]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root = './datasets/muffin/original/'

IMAGE_SIZE = 224
BATCH_SIZE = 64

# ImageDataGenerator 선언 - RGB 값을 0 ~ 1사이로 스케일링만
image_data_generator = ImageDataGenerator(rescale=1./255)

generator = image_data_generator.flow_from_directory(root,
                                                     target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                     batch_size=BATCH_SIZE,
                                                     class_mode='binary')

print(generator.class_indices)

Found 4733 images belonging to 2 classes.
{'chihuahua': 0, 'muffin': 1}


In [58]:
# class_indices의 key: value 순서를 반대로 한 새로운 dict 생성
target_name = {v: k for k, v in generator.class_indices.items()}
target_name

{0: 'chihuahua', 1: 'muffin'}

In [59]:
target_names = []

# 반복문으로 위 dict의 문자열(value)을 target_names에 추가
for target in generator.classes:
    target_names.append(target_name[target])

In [60]:
import pandas as pd

# 경로, target_names(list) 및 target 클래스로 데이터프레임 생성
m_df = pd.DataFrame({'file_paths': generator.filepaths, 'target_names': target_names ,'targets': generator.classes})
m_df

,file_paths,target_names,targets
0,./datasets/muffin/original/chihuahua\chihuahua...,chihuahua,0
1,./datasets/muffin/original/chihuahua\chihuahua...,chihuahua,0
2,./datasets/muffin/original/chihuahua\chihuahua...,chihuahua,0
3,./datasets/muffin/original/chihuahua\chihuahua...,chihuahua,0
4,./datasets/muffin/original/chihuahua\chihuahua...,chihuahua,0
...,...,...,...
4728,./datasets/muffin/original/muffin\muffin995.png,muffin,1
4729,./datasets/muffin/original/muffin\muffin996.png,muffin,1
4730,./datasets/muffin/original/muffin\muffin997.png,muffin,1
4731,./datasets/muffin/original/muffin\muffin998.png,muffin,1


In [61]:
# file_path의 '\\'를 '/'로 대체
m_df.loc[:, 'file_paths'] = m_df.file_paths.apply(lambda x: x.replace('\\', '/'))
m_df

,file_paths,target_names,targets
0,./datasets/muffin/original/chihuahua/chihuahua...,chihuahua,0
1,./datasets/muffin/original/chihuahua/chihuahua...,chihuahua,0
2,./datasets/muffin/original/chihuahua/chihuahua...,chihuahua,0
3,./datasets/muffin/original/chihuahua/chihuahua...,chihuahua,0
4,./datasets/muffin/original/chihuahua/chihuahua...,chihuahua,0
...,...,...,...
4728,./datasets/muffin/original/muffin/muffin995.png,muffin,1
4729,./datasets/muffin/original/muffin/muffin996.png,muffin,1
4730,./datasets/muffin/original/muffin/muffin997.png,muffin,1
4731,./datasets/muffin/original/muffin/muffin998.png,muffin,1


In [62]:
from sklearn.model_selection import train_test_split

# train, validation 데이터 세트 분할
train_images, validation_images, train_targets, validation_targets = \
                train_test_split(m_df.file_paths, m_df.targets, stratify=m_df.targets, test_size=0.2, random_state=124)

print(train_targets.value_counts())
print(validation_targets.value_counts())

targets
0    2047
1    1739
Name: count, dtype: int64
targets
0    512
1    435
Name: count, dtype: int64


In [63]:
# train, validation Dataframe 생성
train_df = m_df.iloc[train_images.index].reset_index(drop=True)
validation_df = m_df.iloc[validation_images.index].reset_index(drop=True)

print(train_df.shape)
print(validation_df.shape)

(3786, 3)
(947, 3)


### Test 데이터 세트 생성

In [64]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root = './datasets/muffin/test/'

IMAGE_SIZE = 224
BATCH_SIZE = 64

# ImageDataGenerator 선언 - RGB 값을 0 ~ 1사이로 스케일링만
image_data_generator = ImageDataGenerator(rescale=1./255)

generator = image_data_generator.flow_from_directory(root,
                                                     target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                     batch_size=BATCH_SIZE,
                                                     class_mode='binary')

print(generator.class_indices)

Found 1184 images belonging to 2 classes.
{'chihuahua': 0, 'muffin': 1}


In [65]:
# class_indices의 key: value 순서를 반대로 한 새로운 dict 생성
target_name = {v: k for k, v in generator.class_indices.items()}
target_name

{0: 'chihuahua', 1: 'muffin'}

In [66]:
target_names = []

# 반복문으로 위 dict의 문자열(value)을 target_names에 추가
for target in generator.classes:
    target_names.append(target_name[target])

In [67]:
import pandas as pd

# 경로, target_names(list) 및 target 클래스로 데이터프레임 생성
test_df = pd.DataFrame({'file_paths': generator.filepaths, 'target_names': target_names ,'targets': generator.classes})
test_df

,file_paths,target_names,targets
0,./datasets/muffin/test/chihuahua\chihuahua1.png,chihuahua,0
1,./datasets/muffin/test/chihuahua\chihuahua10.png,chihuahua,0
2,./datasets/muffin/test/chihuahua\chihuahua100.png,chihuahua,0
3,./datasets/muffin/test/chihuahua\chihuahua101.png,chihuahua,0
4,./datasets/muffin/test/chihuahua\chihuahua102.png,chihuahua,0
...,...,...,...
1179,./datasets/muffin/test/muffin\muffin95.png,muffin,1
1180,./datasets/muffin/test/muffin\muffin96.png,muffin,1
1181,./datasets/muffin/test/muffin\muffin97.png,muffin,1
1182,./datasets/muffin/test/muffin\muffin98.png,muffin,1


In [68]:
# file_path의 '\\'를 '/'로 대체
test_df.loc[:, 'file_paths'] = test_df.file_paths.apply(lambda x: x.replace('\\', '/'))
test_df

,file_paths,target_names,targets
0,./datasets/muffin/test/chihuahua/chihuahua1.png,chihuahua,0
1,./datasets/muffin/test/chihuahua/chihuahua10.png,chihuahua,0
2,./datasets/muffin/test/chihuahua/chihuahua100.png,chihuahua,0
3,./datasets/muffin/test/chihuahua/chihuahua101.png,chihuahua,0
4,./datasets/muffin/test/chihuahua/chihuahua102.png,chihuahua,0
...,...,...,...
1179,./datasets/muffin/test/muffin/muffin95.png,muffin,1
1180,./datasets/muffin/test/muffin/muffin96.png,muffin,1
1181,./datasets/muffin/test/muffin/muffin97.png,muffin,1
1182,./datasets/muffin/test/muffin/muffin98.png,muffin,1


In [69]:
# test_images, test_targets 생성
test_images, test_targets = test_df.file_paths, test_df.targets

In [70]:
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle
import numpy as np
import cv2

IMAGE_SIZE = 224
BATCH_SIZE = 64

# Dataset 클래스(keras의 Sequence 상속)
class Dataset(Sequence):
    def __init__(self, file_paths, targets, batch_size=BATCH_SIZE, aug=None, preprocess=None, shuffle=False):
        self.file_paths = file_paths
        self.targets = targets
        self.batch_size = batch_size
        self.aug = aug
        self.preprocess = preprocess
        self.shuffle = shuffle

        if self.shuffle:
            # shuffle=True일 경우, 매 epoch 종료 시 객체 생성 및 데이터 섞기
            self.on_epoch_end()

    # __len__(): 전체 데이터 건 수에서 batch_size 단위로 나눈 데이터 수 (소수점 올림)
    # ex) 전체 1000 건, batch_size 30 → batch 하나 당 데이터 수는 33.333... 개
    # 이 때, 소수점 아래를 올림해서 1 batch 당 데이터 수를 34개로 설정한다
    def __len__(self, ):
        return int(np.ceil(len(self.targets) / self.batch_size))

    # __getitem__(): batch_size 단위로 이미지 배열과 target 데이터들을 가져온 뒤, 변환한 값 리턴
    # 쉽게 말해 전처리 메소드
    def __getitem__(self, index):
        file_paths_batch = self.file_paths[index * self.batch_size: (index + 1) * self.batch_size]
        targets_batch = self.targets[index * self.batch_size: (index + 1) * self.batch_size]

        # 0으로만 채운(초기화) 4차원 (batch_size, IMAGE_SIZE, IMAGE_SIZE, 3(RGB)) ndarray 할당
        results_batch = np.zeros((file_paths_batch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))

        # 각 이미지 별로 아래의 전처리 실행
        for i in range(file_paths_batch.shape[0]):
            image = cv2.cvtColor(cv2.imread(file_paths_batch[i]), cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

            # aug가 전달되었을 경우, 해당 Augmentation 적용
            if self.aug is not None:
                image = self.aug(image=image)['image']

            # 전처리 함수가 전달되었을 경우, 해당 함수로 image 추가 전처리
            if self.preprocess is not None:
                self.preprocess(image)

            # results_batch의 i번 인덱스에 전처리된 이미지 추가
            results_batch[i] = image

        # 재정의 된 __getitem__() 메소드의 반환값
        # Augmentation 적용(또는 미적용) 된 문제(이미지, results_batch)와 정답(targets_batch) list 반환
        return results_batch, targets_batch

    # __init__() 안 shuffle 하는 함수 정의
    def on_epoch_end(self):
        if self.shuffle:
            self.file_paths, self.targets = shuffle(self.file_paths, self.targets)

In [84]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import MobileNetV2

# 모델 생성 함수
def create_model(model_name='vgg16', verbose=False):
    # 입력받은 model_name에 따라 다른 모델 사용
    input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    
    if model_name == 'vgg16':
        model = VGG16(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'resnet50': # ResNet50, 74.9% ; ResNet50V2, 76.0%
        model = ResNet50V2(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'xception': # Inception을 기초로 한 모델
        model = Xception(input_tensor=input_tensor, include_top=False, weights='imagenet')
    elif model_name == 'mobilenet':
        model = MobileNetV2(input_tensor=input_tensor, include_top=False, weights='imagenet')

    x = model.output

    # Classifier
    # VGG16 이외의 모델은 층 구조가 깊기 때문에 Dropout 사용 
    x = GlobalAveragePooling2D()(x)
    if model_name != 'vgg16':
        x = Dropout(rate=0.5)(x)
    x = Dense(50, activation='relu')(x)
    if model_name != 'vgg16':
        x = Dropout(rate=0.5)(x)
    output = Dense(1, activation='sigmoid', name='output')(x)
    
    model = Model(inputs=input_tensor, outputs=output)
    
    if verbose:
        model.summary()
    
    return model

### 이후 훈련에 사용할 데이터

In [73]:
from tensorflow.keras.applications.mobilenet import preprocess_input as moblienet_preprocess_input
import albumentations as A

# train, validation, test 데이터의 문제(이미지 경로)와 정답(targets)를 변수에 할당
# target 데이터는 get_dummies를 통해 원핫 인코딩
train_file_paths = train_df['file_paths'].values
train_targets = train_df['targets'].values

validation_file_paths = validation_df['file_paths'].values
validation_targets = validation_df['targets'].values

test_file_paths = test_df['file_paths'].values
test_targets = test_df['targets'].values

# Augmentation
aug = A.Compose([
    A.ShiftScaleRotate(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, p=0.5)
])

In [74]:
from tensorflow.keras.applications.mobilenet import preprocess_input as moblienet_preprocess_input

# train, validation, test 데이터 세트 객체 생성
train_dataset = Dataset(train_file_paths,
                        train_targets,
                        batch_size=BATCH_SIZE,
                        aug=aug,
                        preprocess=moblienet_preprocess_input,
                        shuffle=True)

validation_dataset = Dataset(validation_file_paths,
                             validation_targets,
                             batch_size=BATCH_SIZE,
                             preprocess=moblienet_preprocess_input)

test_dataset = Dataset(test_file_paths,
                       test_targets,
                       batch_size=BATCH_SIZE,
                       preprocess=moblienet_preprocess_input)

### 사전 훈련 모델로 fit (Sequence)

In [85]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy, BinaryCrossentropy

# MobileNetV2 모델 생성 후 컴파일링
model = create_model(model_name='mobilenet', verbose=True)
model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['acc'])

C:\Users\User\AppData\Local\Temp\ipykernel_20476\3705052178.py:22: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(input_tensor=input_tensor, include_top=False, weights='imagenet')


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 112, 112, 32)      │             864 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 112, 112, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 112, 112, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 112, 112, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 112, 112, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 112, 112, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 112, 112, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 112, 112, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 113, 113, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 56, 56, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,322,085 (8.86 MB)

 Trainable params: 2,287,973 (8.73 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [86]:
import gc

# Garbage Collecting
gc.collect()

31521

In [87]:
N_EPOCHS = 10

# 모델 훈련
history = model.fit(train_dataset,
                    batch_size=BATCH_SIZE,
                    epochs=N_EPOCHS,
                    validation_data=validation_dataset)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - acc: 0.9062 - loss: 0.2097

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 396s 6s/step - acc: 0.9069 - loss: 0.2084 - val_acc: 0.5407 - val_loss: 30.1814
Epoch 2/10


KeyboardInterrupt: 

#### Sequence만 사용했을 때
- epoch 당 걸리는 시간도 길고, 과적합도 발생함

### Fine Tuning 사용 

In [88]:
# 기존 모델에서 freezing 된 layer 확인
for i, layer in enumerate(model.layers[:-5]):
    layer.trainable = False
    print(i + 1, '.', layer.name, 'trainable:', layer.trainable)

print('\n######### classifier layers ######### ')
for layer in model.layers[-5:]:
    print(layer.name, 'trainable:', layer.trainable)

1 . input_layer_3 trainable: False
2 . Conv1 trainable: False
3 . bn_Conv1 trainable: False
4 . Conv1_relu trainable: False
5 . expanded_conv_depthwise trainable: False
6 . expanded_conv_depthwise_BN trainable: False
7 . expanded_conv_depthwise_relu trainable: False
8 . expanded_conv_project trainable: False
9 . expanded_conv_project_BN trainable: False
10 . block_1_expand trainable: False
11 . block_1_expand_BN trainable: False
12 . block_1_expand_relu trainable: False
13 . block_1_pad trainable: False
14 . block_1_depthwise trainable: False
15 . block_1_depthwise_BN trainable: False
16 . block_1_depthwise_relu trainable: False
17 . block_1_project trainable: False
18 . block_1_project_BN trainable: False
19 . block_2_expand trainable: False
20 . block_2_expand_BN trainable: False
21 . block_2_expand_relu trainable: False
22 . block_2_depthwise trainable: False
23 . block_2_depthwise_BN trainable: False
24 . block_2_depthwise_relu trainable: False
25 . block_2_project trainable: False

In [89]:
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy, BinaryCrossentropy

IMAGE_SIZE = 224
BATCH_SIZE = 64

# Fine Tuning Module
def fine_tune(datas, model_name, aug, preprocess):
    FIRST_EPOCHS = 10
    SECOND_EPOCHS = 10

    # 받은 데이터(Tuple)로 train, validation, test 데이터 분할 
    train_file_paths, train_targets, \
    validation_file_paths, validation_targets, \
    test_file_paths, test_targets = datas

    # train, validation 데이터 세트 객체 생성
    train_dataset = Dataset(train_file_paths,
                        train_targets,
                        batch_size=BATCH_SIZE,
                        aug=aug,
                        preprocess=preprocess,
                        shuffle=True)

    validation_dataset = Dataset(validation_file_paths,
                                 validation_targets,
                                 batch_size=BATCH_SIZE,
                                 preprocess=preprocess)

    # 모델 생성 후 컴파일링
    model = create_model(model_name=model_name, verbose=True)
    model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['acc'])

    # feature extractor(Convolutional Base) layer들을 전부 freeze
    for layer in model.layers[:-5]:
        layer.trainable = False

    # Classifier만 fit
    model.fit(train_dataset,
              batch_size=BATCH_SIZE,
              epochs=FIRST_EPOCHS,
              validation_data=validation_dataset)

    # 부분 freeze - 138 ~ 154번 layer, convolution Base의 최하단 층 2개만 unfreeze
    for layer in model.layers[138:155]:
        layer.trainable = True
        
    model.compile(optimizer=Adam(1e-5), loss=BinaryCrossentropy(), metrics=['acc'])

    # freeze 이후 모델 fit
    history = model.fit(train_dataset,
                        batch_size=BATCH_SIZE,
                        epochs=SECOND_EPOCHS,
                        validation_data=validation_dataset)

    # 모델과 history 반환
    return model, history

In [90]:
import gc

# Garbage Collecting
gc.collect()

17171

In [91]:
# Fine Tuning 함수 사용 - MobileNet 모델 생성 + Fine Tuning
fine_tune((train_file_paths, train_targets, \
           validation_file_paths, validation_targets, \
           test_file_paths, test_targets),
          'mobilenet',
          aug,
          moblienet_preprocess_input)

C:\Users\User\AppData\Local\Temp\ipykernel_20476\3705052178.py:22: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(input_tensor=input_tensor, include_top=False, weights='imagenet')


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 112, 112, 32)      │             864 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 112, 112, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 112, 112, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 112, 112, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 112, 112, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 112, 112, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 112, 112, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 112, 112, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 113, 113, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 56, 56, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,322,085 (8.86 MB)

 Trainable params: 2,287,973 (8.73 MB)

 Non-trainable params: 34,112 (133.25 KB)

Epoch 1/10


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - acc: 0.6059 - loss: 0.7545 - val_acc: 0.7909 - val_loss: 0.4812
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - acc: 0.7404 - loss: 0.5318 - val_acc: 0.8184 - val_loss: 0.4086
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 107s 2s/step - acc: 0.7651 - loss: 0.4960 - val_acc: 0.8279 - val_loss: 0.3873
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - acc: 0.7859 - loss: 0.4566 - val_acc: 0.8722 - val_loss: 0.3481
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - acc: 0.8202 - loss: 0.4122 - val_acc: 0.8701 - val_loss: 0.3374
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - acc: 0.7973 - loss: 0.4364 - val_acc: 0.8733 - val_loss: 0.3134
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - acc: 0.7960 - loss: 0.4346 - val_acc: 0.8786 - val_loss: 0.3261
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - acc: 0.7988 - loss: 0.4394 - val_acc: 0.8680 - val_loss: 0.3057
Epoch 9/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - acc: 0.8172 - loss: 0.4

(<Functional name=functional_9, built=True>,
 <keras.src.callbacks.history.History at 0x21baac3a410>)

### **함수 반환값 변수에 담기!!!**